<a href="https://colab.research.google.com/github/werlang/emolearn-ml-model/blob/main/daisee_eng_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import Sequence, plot_model, to_categorical
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import TimeDistributed, GRU, LSTM, Dropout, Conv1D, Conv2D, Conv3D, ConvLSTM2D, BatchNormalization, MaxPooling1D, MaxPooling2D, MaxPooling3D, GlobalAveragePooling2D, Flatten, Dense, Input, Add, Activation, AveragePooling3D, AveragePooling2D, ZeroPadding3D, Bidirectional, Concatenate
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback, ModelCheckpoint
from tensorflow.keras.metrics import AUC
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import os, cv2
import datetime
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import math
from IPython.display import Image, display
from numba import cuda
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler 
import functools

# from imblearn.over_sampling import SMOTE
# sm = SMOTE(sampling_strategy=0.6666)
# X, y = sm.fit_resample(X, y)


drive_save_path = 'drive/My Drive/1NOSYNC/DT/checkpoint'
labels_path = 'labels'
features_path = 'features'
ident_name = 'daisee-eng-merge'
dir_name = '2021-6-22-12-47-14-daisee-eng-merge'
batch_size = 64
time_frames = 20
interval = 2
stride = 1
fold_step = 1
n_folds = 5

epoch = 0

def restart():
    cuda.select_device(0)
    cuda.close()


def start_colab():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    !pip install Keras-Applications
    # !pip install git+https://github.com/rcmalli/keras-vggface.git
    !pip install keras-tcn


def extract_data():
    !mkdir features

    print("COPYING TRAIN SET...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/daisee_aligned/Train.zip" -d features 
    print("COPYING VALIDATION SET...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/daisee_aligned/Validation.zip" -d features 
    print("COPYING TEST SET...")
    !unzip -n -q "drive/My Drive/1NOSYNC/DT/daisee_aligned/Test.zip" -d features 
    print("COPYING OPENFACE FEATURES...")
    !unzip -n -q "drive/My Drive/Doutorado/Implementação/daisee_openface.zip" -d of_features
    print("COPYING LABELS...")
    !cp -r "drive/My Drive/Doutorado/Implementação/labels" ./

    print("DONE")


def create_folds():
    num_classes = 2
    print("CREATING FOLDS...")
    subs = {}

    Y = []
    clips = []
    # just concatenate all splits
    for split in ["Train", "Validation"]:
        csv_path = "{}/{}Labels.csv".format(labels_path, split)
        csv = pd.read_csv(csv_path)
        C = np.array(csv['ClipID'])
        Yt = np.array(csv.iloc[:,1:])
        if len(Y) == 0:
            clips = C
            Y = Yt
        else:
            Y = np.concatenate((Y, Yt), axis=0)
            clips = np.concatenate((clips, C), axis=0)
    Y = to_categorical(np.array(Y[:,1]) // (4 // num_classes), num_classes)
    

    for i in range(len(clips)):
        file_name = clips[i].split(".")[0]
        subject = int(file_name[:6])
        if not subject in subs:
            subs[subject] = [0 for x in range(num_classes)]
        subs[subject] = np.add(subs[subject], Y[i])

    # total = np.sum(Y, axis=0)
    # print(total, total / len(Y))
    # print(subs)

    folds = []
    fold_names = []
    for i in subs:
        if len(folds) < n_folds:
            folds.append(subs[i])
            fold_names.append([i])
        else:
            summed = np.sum(folds, axis=1)
            index = list(summed).index(min(summed))
            folds[index] = np.add(folds[index], subs[i])
            fold_names[index].append(i)

    # print(folds)
    # print(fold_names)
    # print(np.sum(folds, axis=1))

    # print("GETTING FOLDS DATA...")
    # build frames array
    videos, labels, opface = [], [], []
    for i in range(len(fold_names)):
        videos.append([])
        opface.append([])
        labels.append([])

    fps = 15
    skip = int(round(interval * fps / time_frames, 0))

    split_start = []
    csv_path = "{}/TrainLabels.csv".format(labels_path)
    csv = pd.read_csv(csv_path)
    split_start.append(len(csv))
    csv_path = "{}/ValidationLabels.csv".format(labels_path)
    csv = pd.read_csv(csv_path)
    split_start.append(split_start[0] + len(csv))

    for Yi in range(len(clips)):
        file_name = clips[Yi].split(".")[0]
        subject = file_name[:6]

        for f in range(n_folds):
            if int(subject) in fold_names[f]:
                # print("{}/{}".format(split, subject))
                split = 'Train' if Yi < split_start[0] else 'Validation'
                # get aligned faces file names into files array
                dir_path = "{}/{}/{}/{}_aligned".format(features_path, split, subject, file_name)
                of_path = "of_{}/{}/{}/{}.csv".format(features_path, split, subject, file_name)
                files, filesof = [], []

                if os.path.isfile(of_path):
                    csv_of = pd.read_csv(of_path)
                    rows = csv_of[[' gaze_0_x',' gaze_0_y',' gaze_0_z',' gaze_1_x',' gaze_1_y',' gaze_1_z',' gaze_angle_x',' gaze_angle_y',' pose_Tx',' pose_Ty',' pose_Tz',' pose_Rx',' pose_Ry',' pose_Rz',' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r',' AU10_r',' AU12_r',' AU14_r',' AU15_r',' AU17_r',' AU20_r',' AU23_r',' AU25_r',' AU26_r',' AU45_r',' AU01_c',' AU02_c',' AU04_c',' AU05_c',' AU06_c',' AU07_c',' AU09_c',' AU10_c',' AU12_c',' AU14_c',' AU15_c',' AU17_c',' AU20_c',' AU23_c',' AU25_c',' AU26_c',' AU28_c',' AU45_c']]
                    # rows = csv_of.iloc[:,5:]
                    min_max_scaler = MinMaxScaler()
                    rows = min_max_scaler.fit_transform(rows)
                
                # there are only 150 files in each folder
                for i in range(1,350):
                    image_path = "{}/frame_det_00_000{:03d}.jpg".format(dir_path, i)
                    if os.path.isfile(image_path) and os.path.isfile(of_path):
                        files.append(image_path)
                        filesof.append(rows[i-1])

                # append only the parts relative to the video section
                last = (len(files) - time_frames*skip) // skip
                for i in range(0, last + 1, np.max([1, int(stride * time_frames)])):
                    temp, tpof = [], []
                    for j in range(i*skip, (i+time_frames)*skip, skip):
                        temp.append(files[j])
                        tpof.append(filesof[j])
                    videos[f].append(temp)
                    opface[f].append(tpof)
                    labels[f].append(Y[Yi])

    return videos, opface, labels


def save_folds(folds_data):
    videos, opface, labels = folds_data

    !mkdir folds

    first_only = True

    # ros = RandomOverSampler(random_state=42, sampling_strategy={0: 4000, 1: 31870})
    # rus = RandomUnderSampler(random_state=42, sampling_strategy={0: 4000, 1: 10000})
    for i in range(n_folds):
        print("SAVING FOLD {}...".format(i+1))
        train_X = videos.copy()
        train_XO = opface.copy()
        train_Y = labels.copy()

        val_X = train_X.pop(i)
        val_XO = train_XO.pop(i)
        val_Y = train_Y.pop(i)

        train_X = np.concatenate(train_X)
        train_XO = np.concatenate(train_XO)
        train_Y = np.concatenate(train_Y)

        # create index array for resample
        I = np.array([x for x in range(len(train_X))]).reshape(-1,1)

        # I, train_Y = ros.fit_resample(I, train_Y)
        # I, train_Y = rus.fit_resample(I, train_Y)
        # needed only when 2 classes
        # train_Y = to_categorical(train_Y, 2)

        # copy resampled elements from original arrays according to resampling indexes
        ntx, ntxo = [], []
        for x in I:
            ntx.append(train_X[x[0]])
            ntxo.append(train_XO[x[0]])
        train_X = ntx
        train_XO = ntxo

        # save folds to file
        train_Y = np.array(train_Y)
        np.save("folds/fold_{}_train_Y.npy".format(i), train_Y)
        
        train_X = np.array(train_X)
        np.save("folds/fold_{}_train_XV.npy".format(i), train_X)

        train_XO = np.array(train_XO)
        np.save("folds/fold_{}_train_XO.npy".format(i), train_XO)

        val_Y = np.array(val_Y)
        np.save("folds/fold_{}_validation_Y.npy".format(i), val_Y)
        
        val_X = np.array(val_X)
        np.save("folds/fold_{}_validation_XV.npy".format(i), val_X)

        val_XO = np.array(val_XO)
        np.save("folds/fold_{}_validation_XO.npy".format(i), val_XO)

        if first_only:
            break


def getTest():
    num_classes = 2
    print("SAVING TEST DATA...")
    csv_path = "{}/TestLabels.csv".format(labels_path)
    csv = pd.read_csv(csv_path)

    clips = np.array(csv['ClipID'])
    Y = to_categorical(np.array(csv['Engagement']) // (4 // num_classes), num_classes)

    fps = 15
    skip = int(round(interval * fps / time_frames, 0))

    videos, opface, labels = [], [], []

    for Yi in range(len(clips)):
        file_name = clips[Yi].split(".")[0]
        subject = file_name[:6]

        dir_path = "{}/Test/{}/{}_aligned".format(features_path, subject, file_name)
        of_path = "of_{}/Test/{}/{}.csv".format(features_path, subject, file_name)

        files, filesof = [], []

        if os.path.isfile(of_path):
            csv_of = pd.read_csv(of_path)
            rows = csv_of[[' gaze_0_x',' gaze_0_y',' gaze_0_z',' gaze_1_x',' gaze_1_y',' gaze_1_z',' gaze_angle_x',' gaze_angle_y',' pose_Tx',' pose_Ty',' pose_Tz',' pose_Rx',' pose_Ry',' pose_Rz',' AU01_r',' AU02_r',' AU04_r',' AU05_r',' AU06_r',' AU07_r',' AU09_r',' AU10_r',' AU12_r',' AU14_r',' AU15_r',' AU17_r',' AU20_r',' AU23_r',' AU25_r',' AU26_r',' AU45_r',' AU01_c',' AU02_c',' AU04_c',' AU05_c',' AU06_c',' AU07_c',' AU09_c',' AU10_c',' AU12_c',' AU14_c',' AU15_c',' AU17_c',' AU20_c',' AU23_c',' AU25_c',' AU26_c',' AU28_c',' AU45_c']]
            # rows = csv_of.iloc[:,5:]
            min_max_scaler = MinMaxScaler()
            rows = min_max_scaler.fit_transform(rows)
        
        for i in range(1,350):
            image_path = "{}/frame_det_00_000{:03d}.jpg".format(dir_path, i)
            if os.path.isfile(image_path) and os.path.isfile(of_path):
                files.append(image_path)
                filesof.append(rows[i-1])

        # append only the parts relative to the video section
        last = (len(files) - time_frames*skip) // skip
        for i in range(0, last + 1, np.max([1, int(stride * time_frames)])):
            temp, tpof = [], []
            for j in range(i*skip, (i+time_frames)*skip, skip):
                temp.append(files[j])
                tpof.append(filesof[j])
            videos.append(temp)
            opface.append(tpof)
            labels.append(Y[Yi])

    np.save("folds/test_XV.npy", videos)
    np.save("folds/test_XO.npy", opface)
    np.save("folds/test_Y.npy", labels)


class Generator_V(Sequence):
    def __init__(self, gen_split, batch_size, frames):
        print("BUILDING GENERATOR...")
        self.batch_size = batch_size 
        self.labels, self.videos, = [], []

        file_prefix = "" if gen_split == "Test" else "fold_{}_".format(fold_step-1)
        self.videos = np.load("folds/{}{}_XV.npy".format(file_prefix, gen_split.lower()))
        self.opface = np.load("folds/{}{}_XO.npy".format(file_prefix, gen_split.lower()))
        self.labels = np.load("folds/{}{}_Y.npy".format(file_prefix, gen_split.lower()))

        # self.videos, self.labels = shuffle(self.videos, self.labels, random_state=0)
        print("Done")


    def __len__(self):
        return int(np.ceil(len(self.videos) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_xv = self.videos[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_xo = self.opface[idx * self.batch_size : (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx + 1) * self.batch_size]

        videos, opface = [], []
        for video in batch_xv:
            images = []
            for name in video:
                img = cv2.imread(name)
                images.append(img/255)
            videos.append(np.array(images))

        # videos = np.expand_dims(videos, axis=4)
        videos = np.array(videos)
        opface = np.array(batch_xo)
        label = np.array(batch_y)

        # regressor
        # label = np.array(batch_y).argmax(axis=1) / 3

        return [videos, opface], label
        # return videos, label
        # return opface, label


def build_model(epoch=0, print_model=False):
    from tcn import TCN, tcn_full_summary

    img_size = 224
    of_features = 49

    def model_convlstm(input):
        filters = 256

        x = ConvLSTM2D(filters, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(5e-4), recurrent_regularizer=l2(1e-6), return_sequences=True)(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = ConvLSTM2D(filters, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(5e-4), recurrent_regularizer=l2(1e-6), return_sequences=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = MaxPooling2D(pool_size=3, strides=2, padding='valid')(x)
        x = Flatten()(x)

        return x

    def model_openface(input):
        filters = 1024

        x = TCN(filters)(input)
        
        return x

    def model_inception(input):
        inception = tf.keras.applications.InceptionResNetV2(
            input_shape = (img_size, img_size, 3),
            weights = 'imagenet',
            include_top = False
        )

        for layer in inception.layers:
            layer.trainable = False

        x = TimeDistributed(inception)(input)

        return x

    print("Building model...")
    ########### IF BUILT, MUST DEFINE A NAME TO APPEND TO DIRECTORY NAME ###############
    global ident_name

    ########### IF LOADED, MUST DEFINE DIR NAME AND STARTING EPOCH ############
    global dir_name

    if not epoch:
        input_v = Input(shape=(time_frames, img_size, img_size, 3))
        input_o = Input(shape=(time_frames, of_features))

        m1 = model_inception(input_v)
        m2 = model_convlstm(m1)
        m3 = model_openface(input_o)

        # joint model
        x = Concatenate()([m2, m3])
        
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.8)(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.8)(x)
        x = Dense(2, activation='softmax')(x)

        model = Model([input_v, input_o], x)
        # model = Model(input_v, x)

        model.compile(
            # loss='mse',
            loss='categorical_crossentropy',
            optimizer = SGD(learning_rate=0.0001, momentum=0.9), 
            # optimizer = Adam(learning_rate=1e-5),
            # metrics=[avgacc,'accuracy'])
            # metrics=['mse'])
            metrics=['accuracy'])
    
        t = datetime.datetime.now()
        prefix = str(t.year) +'-'+ str(t.month) +'-'+ str(t.day) +'-'+ str(t.hour) +'-'+ str(t.minute) +'-'+ str(t.second)
        save_dir = "{}/{}-{}".format(drive_save_path, prefix, ident_name)
        os.mkdir(save_dir)
    else:
        file_name = "{:03d}.h5".format(epoch)
        save_dir = "{}/{}".format(drive_save_path, dir_name)
        print("Loading model from {}/{}.".format(save_dir, file_name))
        model = load_model("{}/{}".format(save_dir, file_name), custom_objects={'TCN': TCN})

    if print_model:
        plot_model(model, show_layer_names=False, show_shapes=True, expand_nested=True)
        display(Image('model.png'))

    return model, save_dir, epoch


def set_callbacks():
    #callbacks
    checkpoint = ModelCheckpoint(
        filepath = save_dir + '/{epoch:03d}.h5', 
        monitor = 'val_loss', 
        verbose=1, 
        save_best_only=True,
    )

    tensorboard = TensorBoard(
    	log_dir         = "{}/logs".format(save_dir),
    	histogram_freq  = 0,
    	write_graph     = True,
    	write_grads     = False,
    	write_images    = True
    )

    early_stop = EarlyStopping(
        monitor 	= 'val_loss',
        patience 	= 20,
        restore_best_weights = True,
        verbose     = 1,
        min_delta   = 1e-5
    )

    reduce_lr_plateau = ReduceLROnPlateau(
        monitor 	= 'val_loss',
        factor		= 0.5,
        patience	= 10,
        min_lr		= 1e-6,
        verbose     = 1
    )

    # return [checkpoint, tensorboard]
    return [checkpoint, early_stop, reduce_lr_plateau, tensorboard]


def fit_model(train_weights, epoch=0):
    #calculate weights based on train set distribution
    num_classes = len(train_weights)
    if train_weights == [1 for x in range(num_classes)]:
        weights = {x:1 for x in range(num_classes)}
    else:
        weights = {x: train_weights[x] for x in range(len(train_weights))}
        print("Train weights: {}".format(weights))

    def run():
        #run the model
        return model.fit(
            gen_train,
            epochs = 1000,
            validation_data = gen_val,
            class_weight = weights,
            callbacks = callbacks,
            initial_epoch = epoch)
        
    hist = run()  

    plot_hist(hist)

    return hist


def plot_hist(hist):
    plt.plot(hist.history['loss'], '#0000ff', label="loss")
    plt.plot(hist.history['val_loss'], '#ff0000', label="val_loss")
    plt.legend()
    plt.show()

    plt.plot(hist.history['accuracy'], '#0000ff', label="acc")
    plt.plot(hist.history['val_accuracy'], '#ff0000', label="val_acc")
    # plt.plot(hist.history['accuracy'], '#0055aa', label="acc")
    # plt.plot(hist.history['val_accuracy'], '#aa5500', label="val_acc")
    plt.legend()
    plt.show()


def predict(**kw):
    global gen_test

    Y_true = np.load("folds/test_Y.npy")
    num_classes = Y_true.shape[1]
    Y_true = Y_true.argmax(axis=1)

    print("Predicting...")
    Y_pred = model.predict(gen_test, verbose=1).argmax(axis=1)
    print("\nConfusion matrix:")
    cm = confusion_matrix(Y_true, Y_pred)
    print(cm)

    print("\nEvaluating...")
    ev = model.evaluate(gen_test, verbose=1)
    # print("Loss: {}, Acc: {}".format(ev[0], ev[1]))

    hits = [0 for x in range(num_classes)]
    total = [0 for x in range(num_classes)]
    for i in range(len(Y_true)):
        for c in range(num_classes):
            if (Y_true[i] == c and Y_pred[i] == c):
                hits[c] = hits[c] + 1
            if Y_true[i] == c:
                total[c] = total[c] + 1

    print("\nPer class accuracy:")
    for c in range(num_classes):
        hits[c] = hits[c] / total[c]
        print("{}: {}".format(c, hits[c]))

    print("Average class accuracy: {}".format(sum(hits) / len(hits)))

    f1 = f1_score(Y_true, Y_pred, average=None)
    print("F1 score: {}. Avg F1: {}".format(f1, np.mean(f1)))

    return gen_test
        

In [ ]:
# restart()
# start_colab()
# extract_data()
# folds = create_folds()
# save_folds(folds)
# getTest()

# gen_train = Generator_V('Train', batch_size, time_frames)
# gen_val = Generator_V('Validation', batch_size, time_frames)
# gen_test = Generator_V('Test', batch_size, time_frames)

# model, save_dir, epoch = build_model(epoch, print_model=False)

# callbacks = set_callbacks()
# fit_model(list(len(gen_train.labels) / np.sum(gen_train.labels, axis=0)), epoch)
# gen_test = predict()

# model.summary()

In [ ]:
# restart()
# start_colab()
# extract_data()
# folds = create_folds()
# save_folds(folds)

# gen_train = Generator_V('Train', batch_size, time_frames)
# gen_val = Generator_V('Validation', batch_size, time_frames)
# gen_test = Generator_V('Test', batch_size, time_frames)

# model, save_dir, epoch = build_model(epoch, print_model=True)

callbacks = set_callbacks()
fit_model(list(len(gen_train.labels) / np.sum(gen_train.labels, axis=0)), epoch)
gen_test = predict()

# model.summary()